In trying to predict the outcome of March Madness Games it is first important to figure out the best teams. For this I will be putting together a Value Over Average statistic for each team based on every play of a season. 

Methodology:

Points scored in a close game are more important than a blowout. Points scored in a close game with only a few seconds left are more important than at the very beginning of a game. So with these in mind the value of a play needs to take into account both score and time remaining. 

The baseline value of a play will be points scored:

    Freethrow = 1
    2pter = 2
    3pter = 3

How to categorize defense?

    Blocks, steals, defensive rebounds can be used. Should the defensive team be rewarded for the other team missing? 
    
    There will be an OffenseType and DefenseType parameter for each value. 

Close games should result in higher values

    if deltaScore +- 5 add 10% ?
    
Time remaining should adjust this further. Down by 2 with 5 seconds and you hit a 3 should be extreme value. 
    Start this adjustment with 5 mins left in the game?
    Need to adjust for overtime

A miss should have an equally negative effect on a plays value in high stakes situations:
    under 5 mins with 6 points?


Thoughts for other factors:

Blowing out bad teams is more important than close wins against good teams. 
Do Assists indicate more stable teams?
Do % of points from jump shots indicate anything?


Adjustments:

Need to find ways to adjust for opponent, the conference the team plays in (the better conferences are just simply better)

In [131]:
import pandas as pd
import numpy as np

In [3]:
plays2015 = pd.read_csv('MEvents2015.csv')

In [4]:
plays2015.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0


In [5]:
event_types = plays2015['EventType'].unique()

In [6]:
event_types

array(['miss3', 'reb', 'made2', 'assist', 'turnover', 'steal', 'foul',
       'miss2', 'made3', 'timeout', 'sub', 'made1', 'miss1', 'block'],
      dtype=object)

In [7]:
def adjust_value(base_value, time_bonus, close_game, blown_out):
    if time_bonus == True:
        time_value = base_value * .1 #This can be adjusted but basically it adds value to outcomes in games 
    else:
        time_value = 0
    
    if close_game == True:
        close_value = base_value * .1
    else:
        close_value = 0
    
    adjusted_value = base_value + time_value + close_value
    
    if adjusted_value > 0 and blown_out == True: #If the team is being blown out offense doesnt matter
        adjusted_value = 0
    else:
        pass
    
    return adjusted_value

def get_value(event):
    # Calculate Current Score Differential (WinningTeam - LosingTeam):
    deltaScore = event['WCurrentScore'] - event['LCurrentScore']
    
    #Calculate time remaining and if time bonus is applicable
    time_remaining = 40*60 - event['ElapsedSeconds']
    if time_remaining <= 300:
        time_bonus = True
    else:
        time_bonus = False
        
    #Find out if close game bonus is in effect. This will be deltaScore of 6 or less and 5 mins or less
    if abs(deltaScore) <= 6 and time_bonus == True:
        close_game = True
    else:
        close_game = False
        
    #Find out if team is getting blown out. Check if deltaScore is > 15 and if the team is the winning team or not
    #This will only apply to Offensive Value Additions? 
    if deltaScore > 15 and event['EventTeamId']==event['LTeamID']: #Check if the event team is the team getting blown out
        blown_out = True
    else:
        blown_out = False
        
    #Determine if the event is Offensive or Defensive.
    if ((event['EventType'] == 'reb') & (event['EventSubType']=='def') or (event['EventType'] == 'block') or (event['EventType']=='steal')):
        ValueType = 'Def'
    else:
        ValueType = 'Off'
        
    #Determine base value:
    if event['EventType']=='made1':
        base_value = 1
    elif event['EventType']=='made2':
        base_value = 2
    elif event['EventType']=='made3':
        base_value = 3
    elif event['EventType']=='turnover':
        base_value = -1
    elif event['EventType']=='steal':
        base_value = -1
    elif event['EventType']=='block':
        base_value = -1
    elif (event['EventType'] == 'reb') & (event['EventSubType']=='def'):
        base_value = -1
    else:
        base_value = 0
        
    #Calculate adjusted value
    adjusted_value = adjust_value(base_value, time_bonus, close_game, blown_out)
    
    return adjusted_value

def get_value_type(event):
    #Determine if the event is Offensive or Defensive.
    if ((event['EventType'] == 'reb') & (event['EventSubType']=='def') or (event['EventType'] == 'block') or (event['EventType']=='steal')):
        ValueType = 'Def'
    else:
        ValueType = 'Off'
    return ValueType

#Want to add a gameID to the column for easing grouping
#to make it easier it will simply be made from the DayNum,WTeamID and LTeamID
def make_gameid(event):
    year = event['Season']
    DayNum = event['DayNum']
    wteam = event['WTeamID']
    lteam = event['LTeamID']
    
    game_id = int(str(year)+str(DayNum)+str(wteam)+str(lteam))
    return game_id
        

In [8]:
test_game = plays2015.loc[(plays2015['WTeamID']==1103) & (plays2015['LTeamID']==1420)]

In [9]:
test_game.head(20)

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0
5,6,2015,11,1103,1420,74,57,0,0,72,1420,11784,turnover,unk,0,0,0
6,7,2015,11,1103,1420,74,57,0,0,73,1103,107,steal,NaN,0,0,0
7,8,2015,11,1103,1420,74,57,0,0,75,1420,11803,foul,unk,0,0,0
8,9,2015,11,1103,1420,74,57,0,0,94,1103,92,made2,jump,0,0,0
9,10,2015,11,1103,1420,74,57,0,0,101,1420,11789,made2,lay,0,0,0


In [10]:
test_game['Value'] = test_game.apply(get_value,axis=1)
test_game['ValueType'] = test_game.apply(get_value_type, axis=1)

C:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
test_game.head(20)

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area,Value,ValueType
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0,0.0,Off
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0,-1.0,Def
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0,2.0,Off
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0,0.0,Off
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0,2.0,Off
5,6,2015,11,1103,1420,74,57,0,0,72,1420,11784,turnover,unk,0,0,0,-1.0,Off
6,7,2015,11,1103,1420,74,57,0,0,73,1103,107,steal,NaN,0,0,0,-1.0,Def
7,8,2015,11,1103,1420,74,57,0,0,75,1420,11803,foul,unk,0,0,0,0.0,Off
8,9,2015,11,1103,1420,74,57,0,0,94,1103,92,made2,jump,0,0,0,2.0,Off
9,10,2015,11,1103,1420,74,57,0,0,101,1420,11789,made2,lay,0,0,0,2.0,Off


In [12]:
test_game.groupby(['EventTeamID','ValueType']).sum()

EventID  Season  DayNum  WTeamID  LTeamID  WFinalScore  \
EventTeamID ValueType                                                           
1103        Def           8821   74555     407    40811    52540         2738   
            Off          54353  451360    2464   247072   318080        16576   
1420        Def           8275   64480     352    35296    45440         2368   
            Off          47867  392925    2145   215085   276900        14430   

                       LFinalScore  WCurrentScore  LCurrentScore  \
EventTeamID ValueType                                              
1103        Def               2109              0              0   
            Off              12768              0              0   
1420        Def               1824              0              0   
            Off              11115              0              0   

                       ElapsedSeconds  EventPlayerID  X  Y  Area  Value  
EventTeamID ValueType                                                    
1103        Def                 48159           3345  0  0     0  -37.4  
            Off                293775          32245  0  0     0   63.2  
1420        Def                 44371         365508  0  0     0  -33.2  
            Off                257866        2121994  0  0     0   44.0

In [13]:
plays2015['Value'] = plays2015.apply(get_value,axis=1)
plays2015['ValueType'] = plays2015.apply(get_value_type, axis=1)
plays2015['GameID'] = plays2015.apply(make_gameid, axis=1)

In [14]:
teams2015 = plays2015.groupby(['EventTeamID','ValueType']).sum()

In [15]:
teams2015.tail(10)

EventID    Season  DayNum   WTeamID   LTeamID  \
EventTeamID ValueType                                                      
1460        Def          996091559   1706705   56198   1150627   1178423   
            Off         6640197775  11735360  376132   7836594   8127456   
1461        Def         1408239982   2109705   77757   1484790   1389156   
            Off         7710844050  11275940  424823   7923800   7455259   
1462        Def         1692002457   2512705   93967   1748259   1649185   
            Off        10655545481  15858050  592166  10995041  10418823   
1463        Def         1223323113   2012985   68150   1399594   1305029   
            Off         6830320057  11501620  381964   7958044   7521447   
1464        Def          959158742   1744990   54554   1168973   1231671   
            Off         6018257746  11175190  343649   7435581   7917508   

                       WFinalScore  LFinalScore  WCurrentScore  LCurrentScore  \
EventTeamID ValueType                                                           
1460        Def              59850        51011              0              0   
            Off             416753       353865              0              0   
1461        Def              67794        55968              0              0   
            Off             365690       303358              0              0   
1462        Def              94940        80124              0              0   
            Off             602469       512842              0              0   
1463        Def              67627        58494              0              0   
            Off             393448       339482              0              0   
1464        Def              68545        60003              0              0   
            Off             439299       383501              0              0   

                       ElapsedSeconds  EventPlayerID  X  Y  Area   Value  \
EventTeamID ValueType                                                      
1460        Def               1042589       10583018  0  0     0  -870.8   
            Off               7420355       71888725  0  0     0  1525.6   
1461        Def               1336053       12858766  0  0     0 -1082.8   
            Off               7445355       68633641  0  0     0  1712.8   
1462        Def               1510558       15892368  0  0     0 -1283.2   
            Off              10436864       97936781  0  0     0  2338.0   
1463        Def               1261159       11851510  0  0     0 -1033.0   
            Off               7703357       64525009  0  0     0  1637.2   
1464        Def               1045633        9342041  0  0     0  -888.4   
            Off               7205236       59196753  0  0     0  1768.6   

                                   GameID  
EventTeamID ValueType                      
1460        Def         53705381307448080  
            Off        356229241574076288  
1461        Def         81131690549289184  
            Off        449931611545465920  
1462        Def         94956214184239328  
            Off        592791726560803328  
1463        Def         71277378997245336  
            Off        395965625987966144  
1464        Def         49010267090961712  
            Off        306012189263735744

In [16]:
teams = pd.read_csv('MTeams.csv')

In [17]:
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2020
1,1102,Air Force,1985,2020
2,1103,Akron,1985,2020
3,1104,Alabama,1985,2020
4,1105,Alabama A&M,2000,2020


In [18]:
team_ids = teams2015.index.get_level_values(0).unique()

In [19]:
team_ids = team_ids.drop(0)

In [20]:
team_ids

Int64Index([1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111,
            ...
            1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464],
           dtype='int64', name='EventTeamID', length=351)

In [21]:
def_value = []
off_value = []
total_value = []
for team in team_ids:
    def_value.append(teams2015.xs(team)['Value'][0])
    off_value.append(teams2015.xs(team)['Value'][1])
    total_value.append(off_value[-1] - def_value[-1])
    

In [22]:
df_value = pd.DataFrame(zip(list(team_ids),def_value,off_value,total_value),columns=['TeamID','Def Value','Off Value','Total Value'])

In [23]:
df_value.tail()

,TeamID,Def Value,Off Value,Total Value
346,1460,-870.8,1525.6,2396.4
347,1461,-1082.8,1712.8,2795.6
348,1462,-1283.2,2338.0,3621.2
349,1463,-1033.0,1637.2,2670.2
350,1464,-888.4,1768.6,2657.0


In [24]:
test_game['GameID'] = test_game.apply(make_gameid,axis=1)

C:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
test_game.head()

,EventID,Season,DayNum,WTeamID,LTeamID,WFinalScore,LFinalScore,WCurrentScore,LCurrentScore,ElapsedSeconds,EventTeamID,EventPlayerID,EventType,EventSubType,X,Y,Area,Value,ValueType,GameID
0,1,2015,11,1103,1420,74,57,0,0,19,1103,100,miss3,unk,0,0,0,0.0,Off,20151111031420
1,2,2015,11,1103,1420,74,57,0,0,19,1420,11784,reb,def,0,0,0,-1.0,Def,20151111031420
2,3,2015,11,1103,1420,74,57,0,0,27,1420,11789,made2,dunk,0,0,0,2.0,Off,20151111031420
3,4,2015,11,1103,1420,74,57,0,0,27,1420,11803,assist,NaN,0,0,0,0.0,Off,20151111031420
4,5,2015,11,1103,1420,74,57,0,0,59,1103,87,made2,jump,0,0,0,2.0,Off,20151111031420


In [26]:
games2015 = plays2015.groupby(by=['GameID','EventTeamID','ValueType']).sum()['Value']

In [27]:
games2015.head()

GameID          EventTeamID  ValueType
20151111031420  1103         Def         -37.4
                             Off          63.2
                1420         Def         -33.2
                             Off          44.0
20151111041406  1104         Def         -44.4
Name: Value, dtype: float64

In [28]:
games2015[20151111031420,1103,'Def']

-37.400000000000006

In [29]:
test_game_id = 20151111031420

In [30]:
games2015.index.get_level_values(0)

Int64Index([ 20151111031420,  20151111031420,  20151111031420,  20151111031420,
             20151111041406,  20151111041406,  20151111041406,  20151111041406,
             20151111121291,  20151111121291,
            ...
            201515013901274, 201515013901274, 201515211811277, 201515211811277,
            201515211811277, 201515211811277, 201515214581246, 201515214581246,
            201515214581246, 201515214581246],
           dtype='int64', name='GameID', length=21924)

In [31]:
#Get the opponent teamID from just gameID
a = games2015.loc[20151111031420].index.get_level_values(0).unique()
a

Int64Index([1103, 1420], dtype='int64', name='EventTeamID')

In [32]:
b = [i for i in a if i != 1390][0]
b

1103

In [33]:
games2015.loc[20151111031420,b,'Off']

63.2

In [34]:
team_ids

Int64Index([1101, 1102, 1103, 1104, 1105, 1106, 1107, 1108, 1110, 1111,
            ...
            1455, 1456, 1457, 1458, 1459, 1460, 1461, 1462, 1463, 1464],
           dtype='int64', name='EventTeamID', length=351)

In [35]:
def get_opponent_avgs(team_id):
    game_ids = games2015[:,team_id,'Def'].index
    num_games = len(game_ids) #This will be useful
    
    opponent_off_value = []
    opponent_def_value = []
    
    for game in game_ids:
        opponent = [i for i in games2015.loc[game].index.get_level_values(0).unique() if i != team_id][0]
        opponent_off_value.append(games2015[game,opponent,'Off'])
        opponent_def_value.append(games2015[game,opponent,'Def'])
        
    opponent_avg_off_value = sum(opponent_off_value) / len(opponent_off_value)
    opponent_avg_def_value = sum(opponent_def_value) / len(opponent_def_value)
    
    df_info = pd.DataFrame([[team_id,num_games,opponent_avg_off_value,opponent_avg_def_value]],columns=['TeamID','Games Played','Opponents Avg Off Value','Opponents Avg Def Value'])
    
    return df_info

df_opponents_avgs = pd.DataFrame([],columns=['TeamID','Games Played','Opponents Avg Off Value','Opponents Avg Def Value'])

for team in team_ids:
    df_opponents_avgs = df_opponents_avgs.append(get_opponent_avgs(team),ignore_index=True)

In [36]:
get_opponent_avgs(1390)

,TeamID,Games Played,Opponents Avg Off Value,Opponents Avg Def Value
0,1390,37,57.8,-31.854054


In [37]:
df_opponents_avgs.head()

,TeamID,Games Played,Opponents Avg Off Value,Opponents Avg Def Value
0,1101,28,60.178571,-36.900000
1,1102,29,55.372414,-31.379310
2,1103,34,52.629412,-34.917647
3,1104,33,54.345455,-30.442424
4,1105,28,58.742857,-33.757143


In [38]:
df_value.head()

,TeamID,Def Value,Off Value,Total Value
0,1101,-770.0,1389.0,2159.0
1,1102,-868.2,1596.0,2464.2
2,1103,-1211.8,1929.2,3141.0
3,1104,-1120.6,1885.6,3006.2
4,1105,-849.6,1363.6,2213.2


In [39]:
#Now need to adjust the value a team generated in a game to the average of the opponent

def adjusted_value(team_id):
    game_ids = games2015[:,team_id,'Def'].index
    num_games = len(game_ids) #This will be useful
    
    adjusted_off_value = []
    adjusted_def_value = []
    
    for game in game_ids:
        opponent = [i for i in games2015[game].index.get_level_values(0).unique() if i != team_id][0] #This gives the opponents id for the game
        
        adjusted_off_value.append(games2015[game,team_id,'Off'] - df_opponents_avgs.loc[df_opponents_avgs['TeamID']==opponent]['Opponents Avg Off Value'].reset_index(drop=True)[0])
        adjusted_def_value.append(games2015[game,team_id,'Def'] - df_opponents_avgs.loc[df_opponents_avgs['TeamID']==opponent]['Opponents Avg Def Value'].reset_index(drop=True)[0])

    adjusted_off_value = sum(adjusted_off_value) / len(adjusted_off_value)
    adjusted_def_value = sum(adjusted_def_value) / len(adjusted_def_value)
    
    total_adjusted_mean_value = adjusted_off_value - adjusted_def_value
    
    df_info = pd.DataFrame([[team_id,num_games,adjusted_off_value,adjusted_def_value,total_adjusted_mean_value]],columns=['TeamID','Number of Games','Average Adjusted Off Value','Average Adjusted Def Value','Total Average Value'])
    
    return df_info

df_avgs = pd.DataFrame([],columns=['TeamID','Number of Games','Average Adjusted Off Value','Average Adjusted Def Value','Total Average Value'])

for team in team_ids:
    df_avgs = df_avgs.append(adjusted_value(team),ignore_index=True)

In [40]:
df_avgs.head()

,TeamID,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value
0,1101,28,-9.281126,6.700034,-15.981160
1,1102,29,0.110310,3.613836,-3.503526
2,1103,34,-0.643615,-1.324789,0.681174
3,1104,33,2.575164,-0.681135,3.256298
4,1105,28,-8.160042,4.921737,-13.081779


In [41]:
#Combine this with the teams df
teams.head()

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2020
1,1102,Air Force,1985,2020
2,1103,Akron,1985,2020
3,1104,Alabama,1985,2020
4,1105,Alabama A&M,2000,2020


In [42]:
test = teams.set_index('TeamID').join(df_avgs.set_index('TeamID'))

In [43]:
test.sort_values(by='Total Average Value',ascending=False).head(40)

,TeamName,FirstD1Season,LastD1Season,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value
TeamID,,,,,,,
1140,BYU,1985,2020,33,16.711056,-4.047945,20.759001
1314,North Carolina,1985,2020,38,13.759795,-6.125518,19.885313
1181,Duke,1985,2020,38,15.412359,-4.460092,19.872451
1235,Iowa St,1985,2020,34,15.077387,-4.166219,19.243606
1323,Notre Dame,1985,2020,38,15.443871,-2.363674,17.807545
1246,Kentucky,1985,2020,39,10.907812,-6.658158,17.565970
1211,Gonzaga,1985,2020,36,12.679035,-4.492412,17.171447
1233,Iona,1985,2020,35,13.465030,-2.706754,16.171784
1112,Arizona,1985,2020,38,11.231602,-4.304155,15.535757


In [44]:
data = teams.set_index('TeamID').join(df_value.set_index('TeamID'))
#By leaving it as total's for the value column the analysis rewards teams that played more games = conference tourney success

In [45]:
data=data.join(df_avgs.set_index('TeamID'))
data.drop(['FirstD1Season','LastD1Season'],axis=1,inplace=True)
data.sort_values(by='Off Value',ascending=False).head(30)


,TeamName,Def Value,Off Value,Total Value,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value
TeamID,,,,,,,,
1181,Duke,-1419.8,2684.0,4103.8,38,15.412359,-4.460092,19.872451
1323,Notre Dame,-1351.4,2679.4,4030.8,38,15.443871,-2.363674,17.807545
1458,Wisconsin,-1265.8,2622.2,3888.0,39,12.331850,0.644317,11.687533
1112,Arizona,-1429.6,2568.6,3998.2,38,11.231602,-4.304155,15.535757
1246,Kentucky,-1551.8,2556.0,4107.8,39,10.907812,-6.658158,17.565970
1314,North Carolina,-1462.6,2549.2,4011.8,38,13.759795,-6.125518,19.885313
1211,Gonzaga,-1355.2,2486.6,3841.8,36,12.679035,-4.492412,17.171447
1437,Villanova,-1324.4,2440.6,3765.0,36,11.676686,-2.817729,14.494414
1116,Arkansas,-1299.6,2433.4,3733.0,36,12.542857,-2.794523,15.337381


In [46]:
#With the value stats completed (can still look at adjusting the parameters) now can began completing some season total stats. W,L,windiff etc.

stats = pd.read_csv('MRegularSeasonDetailedResults.csv')

In [47]:
stats.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


In [48]:
def get_year_stats(year):
    stats_year = stats.loc[stats['Season']==year]
    
    avg_win_diffs = []
    avg_game_diffs = []
    num_wins = []
    num_losses = []
    for team in team_ids:
        #Find the average WinDiff for each team
        team_wins = stats_year.loc[stats_year['WTeamID']==team]
        team_avg_win_diff = (team_wins['WScore'].sum() - team_wins['LScore'].sum()) / len(team_wins['WScore'])
        avg_win_diffs.append(round(team_avg_win_diff,2))
        
        #Get total average game diff
        
        team_losses = stats_year.loc[stats_year['LTeamID']==team]
        team_avg_game_diff = (team_wins['WScore'].sum() + team_losses['LScore'].sum() - team_wins['LScore'].sum() - team_losses['WScore'].sum()) / len(team_wins['WScore'])
        avg_game_diffs.append(round(team_avg_game_diff,2))
        
        #Team counting stats
        num_wins.append(len(team_wins))
        num_losses.append(len(team_losses))
        
        
    df_info = pd.DataFrame(zip(team_ids, [year]*len(num_wins), num_wins, num_losses, avg_win_diffs, avg_game_diffs),columns=['TeamID','Year','Wins','Losses','Avg Win Diff','Avg Game Diff'])
        
        
    return df_info

In [49]:
df_win_diffs = get_year_stats(2015)

C:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: invalid value encountered in longlong_scalars
  # This is added back by InteractiveShellApp.init_path()
C:\Programming\Anaconda\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in longlong_scalars


In [50]:
data=data.join(df_win_diffs.set_index('TeamID'))

In [51]:
data.sort_values(by='Avg Game Diff',ascending=False).head(40)

,TeamName,Def Value,Off Value,Total Value,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Year,Wins,Losses,Avg Win Diff,Avg Game Diff
TeamID,,,,,,,,,,,,,
1246,Kentucky,-1551.8,2556.0,4107.8,39,10.907812,-6.658158,17.565970,2015.0,34.0,0.0,20.94,20.94
1112,Arizona,-1429.6,2568.6,3998.2,38,11.231602,-4.304155,15.535757,2015.0,31.0,3.0,19.84,19.55
1428,Utah,-1255.0,2083.2,3338.2,34,4.450880,-3.348267,7.799147,2015.0,23.0,8.0,22.30,19.52
1326,Ohio St,-1331.0,2302.8,3633.8,35,10.445731,-4.665523,15.111254,2015.0,23.0,10.0,22.74,19.26
1211,Gonzaga,-1355.2,2486.6,3841.8,36,12.679035,-4.492412,17.171447,2015.0,31.0,2.0,18.42,18.23
1458,Wisconsin,-1265.8,2622.2,3888.0,39,12.331850,0.644317,11.687533,2015.0,31.0,3.0,17.97,17.29
1181,Duke,-1419.8,2684.0,4103.8,38,15.412359,-4.460092,19.872451,2015.0,29.0,4.0,18.55,17.10
1437,Villanova,-1324.4,2440.6,3765.0,36,11.676686,-2.817729,14.494414,2015.0,32.0,2.0,17.12,16.34
1438,Virginia,-1239.2,1974.4,3213.6,34,2.410093,-2.958237,5.368330,2015.0,29.0,3.0,16.52,16.10


In [52]:
#Try and grab the Kenpom ratings #Note that for previous years this will include the tourney games

In [53]:
kenpom2015 = pd.read_csv('2015Kenpom.csv')

In [54]:
kenpom2015.head()

,TeamName,Kenpom Overall,Kenpom Tempo,Kenpom RankAdjOE,Kenpom RankAdjDE
0,Kentucky,1,251,5,2
1,Arizona,2,78,11,3
2,Wisconsin,3,347,1,30
3,Virginia,4,349,27,1
4,Villanova,5,181,4,13


In [55]:
# Check for any errors in the kenpom document
for name in kenpom2015['TeamName']:
    dict_fix = {}
    try:
        team_id = data.loc[data['TeamName']==name.strip('.')].index[0]
    except IndexError:
        print('{} is not in the database'.format(name))
        correct = input('What is the correct spelling')        
        dict_fix[name] = correct


In [56]:
data.loc[data['TeamName']=='Arizona']

,TeamName,Def Value,Off Value,Total Value,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Year,Wins,Losses,Avg Win Diff,Avg Game Diff
TeamID,,,,,,,,,,,,,
1112,Arizona,-1429.6,2568.6,3998.2,38,11.231602,-4.304155,15.535757,2015.0,31.0,3.0,19.84,19.55


In [57]:
kenpom2015.loc[kenpom2015['TeamName']=='Arizona']

,TeamName,Kenpom Overall,Kenpom Tempo,Kenpom RankAdjOE,Kenpom RankAdjDE
1,Arizona,2,78,11,3


In [58]:
kenpom2015.head()

,TeamName,Kenpom Overall,Kenpom Tempo,Kenpom RankAdjOE,Kenpom RankAdjDE
0,Kentucky,1,251,5,2
1,Arizona,2,78,11,3
2,Wisconsin,3,347,1,30
3,Virginia,4,349,27,1
4,Villanova,5,181,4,13


In [59]:
def get_team_id(team):
    try:
        team_data = pd.read_csv('MTeams.csv')
        team_id = team_data.loc[team_data['TeamName']==team]['TeamID'].values[0]
        return team_id
    except IndexError:
        print(team)

In [60]:
kenpom2015['TeamID'] = kenpom2015.apply(lambda team: get_team_id(team['TeamName'].strip('.')),axis=1)

In [61]:
kenpom2015.set_index('TeamID',inplace=True)

In [62]:
#Join Kenpom and data dataframes
df_all_2015 = data.join(kenpom2015.drop('TeamName',axis=1))

In [63]:
kenpom2015.head()


,TeamName,Kenpom Overall,Kenpom Tempo,Kenpom RankAdjOE,Kenpom RankAdjDE
TeamID,,,,,
1246,Kentucky,1,251,5,2
1112,Arizona,2,78,11,3
1458,Wisconsin,3,347,1,30
1438,Virginia,4,349,27,1
1437,Villanova,5,181,4,13


In [64]:
data.head()

,TeamName,Def Value,Off Value,Total Value,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Year,Wins,Losses,Avg Win Diff,Avg Game Diff
TeamID,,,,,,,,,,,,,
1101,Abilene Chr,-770.0,1389.0,2159.0,28,-9.281126,6.700034,-15.981160,2015.0,7.0,21.0,12.14,-43.43
1102,Air Force,-868.2,1596.0,2464.2,29,0.110310,3.613836,-3.503526,2015.0,12.0,17.0,13.67,-2.75
1103,Akron,-1211.8,1929.2,3141.0,34,-0.643615,-1.324789,0.681174,2015.0,20.0,14.0,11.90,7.00
1104,Alabama,-1120.6,1885.6,3006.2,33,2.575164,-0.681135,3.256298,2015.0,17.0,14.0,10.35,3.88
1105,Alabama A&M,-849.6,1363.6,2213.2,28,-8.160042,4.921737,-13.081779,2015.0,8.0,20.0,9.00,-26.00


In [65]:
df_all_2015.sort_values(by='Total Average Value',ascending=False).head(10)

,TeamName,Def Value,Off Value,Total Value,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Year,Wins,Losses,Avg Win Diff,Avg Game Diff,Kenpom Overall,Kenpom Tempo,Kenpom RankAdjOE,Kenpom RankAdjDE
TeamID,,,,,,,,,,,,,,,,,
1140,BYU,-1215.8,2394.8,3610.6,33,16.711056,-4.047945,20.759001,2015.0,23.0,9.0,14.74,12.39,31.0,6.0,9.0,139.0
1314,North Carolina,-1462.6,2549.2,4011.8,38,13.759795,-6.125518,19.885313,2015.0,24.0,11.0,17.67,13.88,16.0,10.0,12.0,45.0
1181,Duke,-1419.8,2684.0,4103.8,38,15.412359,-4.460092,19.872451,2015.0,29.0,4.0,18.55,17.10,7.0,103.0,3.0,57.0
1235,Iowa St,-1283.2,2338.8,3622.0,34,15.077387,-4.166219,19.243606,2015.0,25.0,8.0,14.12,12.00,13.0,15.0,7.0,82.0
1323,Notre Dame,-1351.4,2679.4,4030.8,38,15.443871,-2.363674,17.807545,2015.0,29.0,5.0,17.03,15.45,10.0,204.0,2.0,112.0
1246,Kentucky,-1551.8,2556.0,4107.8,39,10.907812,-6.658158,17.565970,2015.0,34.0,0.0,20.94,20.94,1.0,251.0,5.0,2.0
1211,Gonzaga,-1355.2,2486.6,3841.8,36,12.679035,-4.492412,17.171447,2015.0,31.0,2.0,18.42,18.23,6.0,153.0,6.0,20.0
1233,Iona,-1304.6,2419.6,3724.2,35,13.465030,-2.706754,16.171784,2015.0,26.0,8.0,11.38,8.88,108.0,17.0,39.0,233.0
1112,Arizona,-1429.6,2568.6,3998.2,38,11.231602,-4.304155,15.535757,2015.0,31.0,3.0,19.84,19.55,2.0,78.0,11.0,3.0


df_all_2015 contains all the data from each team in one place for the 2015 season. Now to set up a method to generate all possible matchups in the 2015 tourney and simulate the outcomes

In [66]:
df = pd.read_csv('MNCAATourneyCompactResults.csv')

In [67]:
df_2015 = df.loc[df['Season']==2015]

In [68]:
df_2015.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1916,2015,134,1214,74,1264,64,N,0
1917,2015,134,1279,94,1140,90,N,0
1918,2015,135,1173,56,1129,55,N,0
1919,2015,135,1352,81,1316,77,N,0
1920,2015,136,1112,93,1411,72,N,0


Need to get a list of all the teams in the 2015 tourney. (Will then repeat for 2016-2019)

In [69]:
teams_2015 = list(df_2015['WTeamID'].unique())
teams_2015 = teams_2015 + list(df_2015['LTeamID'].unique())
teams_2015 = list(set(teams_2015))

Now need to create a dataframe of matchup data to train a model on

In [70]:
import itertools
teams_2015.sort()

In [71]:
matchups = list(itertools.combinations(teams_2015,2))

In [72]:
def get_stats(team_id, dataframe):
    team_data = dataframe.loc[team_id][['Number of Games','Average Adjusted Off Value','Average Adjusted Def Value','Total Average Value','Wins','Avg Win Diff','Kenpom Overall','Kenpom RankAdjOE','Kenpom RankAdjDE']]
    return team_data

def matchup_data_gen(teamID_1, teamID_2, dataframe):
    try:
        team_1_data = get_stats(teamID_1, dataframe)
        team_1_data['TeamID'] = team_1_data.name

        team_2_data = get_stats(teamID_2, dataframe)
        team_2_data['TeamID'] = team_2_data.name
        team_2_data = team_2_data.add_prefix('Opp_')

        data = team_1_data.append(team_2_data).to_frame().T

        return data
    except AttributeError:
        print(teamID_1, teamID_2)


For each possible matchup generate a row for the dataframe

In [73]:
df_2015_matchups = pd.DataFrame()
for matchup in matchups:
    data = matchup_data_gen(matchup[0],matchup[1],df_all_2015)
    df_2015_matchups = pd.concat([df_2015_matchups, data])

In [135]:
df_2015_matchups.head(20)

,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Wins,Avg Win Diff,Kenpom Overall,Kenpom RankAdjOE,Kenpom RankAdjDE,TeamID,Opp_Number of Games,Opp_Average Adjusted Off Value,Opp_Average Adjusted Def Value,Opp_Total Average Value,Opp_Wins,Opp_Avg Win Diff,Opp_Kenpom Overall,Opp_Kenpom RankAdjOE,Opp_Kenpom RankAdjDE,Opp_TeamID
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,38,11.2316,-4.30416,15.5358,31,19.84,2,11,3,1112
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,36,12.5429,-2.79452,15.3374,26,13.23,29,21,81,1116
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,33,5.056,-3.30846,8.36446,23,15,15,13,33,1124
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,32,5.236,1.23114,4.00486,21,9.81,149,64,259,1125
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,32,6.35873,-0.0764212,6.43515,23,14.74,39,48,64,1129
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,33,9.59144,-4.38215,13.9736,23,12.43,54,47,94,1138
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,34,3.21928,-1.38957,4.60885,22,16,23,67,10,1139
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,33,16.7111,-4.04795,20.759,23,14.74,31,9,139,1140
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,34,-1.79315,-2.66275,0.869602,22,13.91,34,76,18,1153
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,30,1.18736,-0.642344,1.8297,20,12.6,148,136,164,1157


Now to add a column indicating if the teams met in the 2015 tournement and if Team1 won.

In [107]:
df_2015_tourney_results = pd.read_csv('MNCAATourneyCompactResults.csv')

In [109]:
df_2015_tourney_results = df_2015_tourney_results.loc[df_2015_tourney_results['Season']==2015]

In [143]:
df_2015_tourney_results.head(50)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
1916,2015,134,1214,74,1264,64,N,0
1917,2015,134,1279,94,1140,90,N,0
1918,2015,135,1173,56,1129,55,N,0
1919,2015,135,1352,81,1316,77,N,0
1920,2015,136,1112,93,1411,72,N,0
1921,2015,136,1116,56,1459,53,N,0
1922,2015,136,1139,56,1400,48,N,0
1923,2015,136,1153,66,1345,65,N,1
1924,2015,136,1207,84,1186,74,N,0
1925,2015,136,1209,57,1124,56,N,0


In [111]:
Winners = df_2015_tourney_results.WTeamID.values
Losers = df_2015_tourney_results.LTeamID.values

In [112]:
games = list(zip(Winners,Losers))

In [116]:
games_2015 = [(min(game),max(game)) for game in games]

In [139]:
def win_lose(team1, team2):
    
    if len(df_2015_tourney_results.loc[(df_2015_tourney_results['WTeamID'] == team1) & (df_2015_tourney_results['LTeamID']==team2)]) == 1:
        result = 1
    elif len(df_2015_tourney_results.loc[(df_2015_tourney_results['WTeamID'] == team2) & (df_2015_tourney_results['LTeamID']==team1)]) == 1:
        result = 0
    else:
        result = np.nan
    return result

# for game in games_2015:
#     win_lose(game)       
    
df_2015_matchups['Team Win'] = df_2015_matchups.apply(lambda x: win_lose(x['TeamID'],x['Opp_TeamID']),axis=1)

In [128]:
test = df_2015_tourney_results.loc[(df_2015_tourney_results['WTeamID'] == 1214) & (df_2015_tourney_results['LTeamID']==1264)]

In [145]:
df_2015_matchups['Year'] = 2015
df_2015_matchups.head()

,Number of Games,Average Adjusted Off Value,Average Adjusted Def Value,Total Average Value,Wins,Avg Win Diff,Kenpom Overall,Kenpom RankAdjOE,Kenpom RankAdjDE,TeamID,...,Opp_Average Adjusted Def Value,Opp_Total Average Value,Opp_Wins,Opp_Avg Win Diff,Opp_Kenpom Overall,Opp_Kenpom RankAdjOE,Opp_Kenpom RankAdjDE,Opp_TeamID,Team Win,Year
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,...,-4.30416,15.5358,31,19.84,2,11,3,1112,NaN,2015
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,...,-2.79452,15.3374,26,13.23,29,21,81,1116,NaN,2015
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,...,-3.30846,8.36446,23,15,15,13,33,1124,NaN,2015
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,...,1.23114,4.00486,21,9.81,149,64,259,1125,NaN,2015
0,33,-0.0932652,3.01816,-3.11143,24,10.12,131,117,155,1107,...,-0.0764212,6.43515,23,14.74,39,48,64,1129,NaN,2015


In [144]:
len(df_2015_matchups)

2278